In [43]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os
from omegaconf import OmegaConf
from pprint import pprint as pp
from icecream import ic
from typing import List

project = Path().resolve().parent
configs = Path(project) / "config"
data_paths = OmegaConf.load(configs / "data.yaml")
print(OmegaConf.to_yaml(data_paths))

raw: ../data/raw
interim: ../data/interim
processed: ../data/processed



In [44]:
data_int = Path(data_paths.processed) / "modsec_audit.csv"
df = pd.read_csv(data_int, on_bad_lines="warn", engine="python")

In [45]:
df

,http_version,id,layer_type,local_ip,local_port,method,reference,remote_ip,remote_port,req_accept,...,ssrf_value,status_code,target,targeted_filed,timestamp,txn_id,uri,xss,xss_severity,xss_value
0,1.1,XD10tCwH,SINGLE_LAYERED,127.0.0.1,80,OPTIONS,NaN,127.0.0.1,39486,*/*,...,NaN,204,ssrf,NaN,05/May/2025:09:08:24 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
1,1.1,sRNMSqLX,SINGLE_LAYERED,127.0.0.1,80,POST,"v740,50",127.0.0.1,39486,"application/json, text/plain, */*",...,NaN,401,ssrf,NaN,05/May/2025:09:08:25 +0000,1.746436e+10,/api/v1/auths/login,False,NaN,NaN
2,1.1,wWRXeQtO,SINGLE_LAYERED,127.0.0.1,80,OPTIONS,NaN,127.0.0.1,39486,*/*,...,NaN,204,ssrf,NaN,05/May/2025:09:09:36 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
3,1.1,bg2mL7Uq,SINGLE_LAYERED,127.0.0.1,80,POST,"v740,57",127.0.0.1,39486,"application/json, text/plain, */*",...,NaN,200,ssrf,NaN,05/May/2025:09:09:37 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
4,1.1,KTqEfmTV,SINGLE_LAYERED,127.0.0.1,80,GET,NaN,127.0.0.1,39486,*/*,...,NaN,200,ssrf,NaN,05/May/2025:09:09:37 +0000,1.746436e+11,/api/v1/books?search=&page=1&size=9,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20003,1.1,zGcNswkf,NaN,127.0.0.1,1234,POST,NaN,127.0.0.1,47098,"application/json, text/plain, */*",...,NaN,401,NaN,NaN,27/May/2025:04:48:41 +0000,1.748321e+11,/api/v1/auths/login,NaN,NaN,NaN
20004,1.1,qYpLWOle,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,10o2,2o11,1v14,24o48,10o48,2o57,1v740,72",127.0.0.1,58032,"application/json, text/plain, */*",...,NaN,403,sql_injection,NaN,27/May/2025:04:49:25 +0000,1.748321e+11,/api/v1/auths/login,False,NaN,NaN
20005,1.1,noGe56be,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,12o2,2o13,1v14,26o48,12o48,2o59,1v740,74",127.0.0.1,39592,"application/json, text/plain, */*",...,NaN,200,sql_injection,NaN,27/May/2025:05:02:52 +0000,1.748322e+11,/api/v1/auths/login,False,NaN,NaN
20006,1.1,aslSQh2t,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,12o2,2o13,1v14,26o48,12o48,2o59,1v740,74",127.0.0.1,39592,"application/json, text/plain, */*",...,NaN,200,sql_injection,NaN,27/May/2025:05:04:41 +0000,1.748322e+11,/api/v1/auths/login,False,NaN,NaN


In [47]:
# List out important columns to be used
DEFAULT_KEEP_COLS: List[str] = [
    "id",
    "timestamp",
    "layer_type",
    "method",
    "local_port",
    "remote_port",
    "req_user_agent"
    "req_content_length",
    "resp_content_length",
    "resp_vary",
    "status_code",
    "target"
]

In [48]:
df.columns = df.columns.str.strip()
df = df.rename(columns=str.lower)
df

,http_version,id,layer_type,local_ip,local_port,method,reference,remote_ip,remote_port,req_accept,...,ssrf_value,status_code,target,targeted_filed,timestamp,txn_id,uri,xss,xss_severity,xss_value
0,1.1,XD10tCwH,SINGLE_LAYERED,127.0.0.1,80,OPTIONS,NaN,127.0.0.1,39486,*/*,...,NaN,204,ssrf,NaN,05/May/2025:09:08:24 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
1,1.1,sRNMSqLX,SINGLE_LAYERED,127.0.0.1,80,POST,"v740,50",127.0.0.1,39486,"application/json, text/plain, */*",...,NaN,401,ssrf,NaN,05/May/2025:09:08:25 +0000,1.746436e+10,/api/v1/auths/login,False,NaN,NaN
2,1.1,wWRXeQtO,SINGLE_LAYERED,127.0.0.1,80,OPTIONS,NaN,127.0.0.1,39486,*/*,...,NaN,204,ssrf,NaN,05/May/2025:09:09:36 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
3,1.1,bg2mL7Uq,SINGLE_LAYERED,127.0.0.1,80,POST,"v740,57",127.0.0.1,39486,"application/json, text/plain, */*",...,NaN,200,ssrf,NaN,05/May/2025:09:09:37 +0000,1.746436e+11,/api/v1/auths/login,False,NaN,NaN
4,1.1,KTqEfmTV,SINGLE_LAYERED,127.0.0.1,80,GET,NaN,127.0.0.1,39486,*/*,...,NaN,200,ssrf,NaN,05/May/2025:09:09:37 +0000,1.746436e+11,/api/v1/books?search=&page=1&size=9,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20003,1.1,zGcNswkf,NaN,127.0.0.1,1234,POST,NaN,127.0.0.1,47098,"application/json, text/plain, */*",...,NaN,401,NaN,NaN,27/May/2025:04:48:41 +0000,1.748321e+11,/api/v1/auths/login,NaN,NaN,NaN
20004,1.1,qYpLWOle,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,10o2,2o11,1v14,24o48,10o48,2o57,1v740,72",127.0.0.1,58032,"application/json, text/plain, */*",...,NaN,403,sql_injection,NaN,27/May/2025:04:49:25 +0000,1.748321e+11,/api/v1/auths/login,False,NaN,NaN
20005,1.1,noGe56be,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,12o2,2o13,1v14,26o48,12o48,2o59,1v740,74",127.0.0.1,39592,"application/json, text/plain, */*",...,NaN,200,sql_injection,NaN,27/May/2025:05:02:52 +0000,1.748322e+11,/api/v1/auths/login,False,NaN,NaN
20006,1.1,aslSQh2t,SINGLE_LAYERED,127.0.0.1,1234,POST,"o2,12o2,2o13,1v14,26o48,12o48,2o59,1v740,74",127.0.0.1,39592,"application/json, text/plain, */*",...,NaN,200,sql_injection,NaN,27/May/2025:05:04:41 +0000,1.748322e+11,/api/v1/auths/login,False,NaN,NaN


In [51]:
keep = list()
cols_to_keep = keep or DEFAULT_KEEP_COLS
cols_in_file = [c for c in cols_to_keep if c in df.columns]
df = df[cols_in_file].copy()
df.to_csv(Path(data_paths.interim) / "modsec_audit_train_v1.csv")
df.head()

,id,timestamp,layer_type,method,local_port,remote_port,resp_content_length,resp_vary,status_code,target
0,XD10tCwH,05/May/2025:09:08:24 +0000,SINGLE_LAYERED,OPTIONS,80,39486,NaN,"Access-Control-Request-Method, Access-Control-...",204,ssrf
1,sRNMSqLX,05/May/2025:09:08:25 +0000,SINGLE_LAYERED,POST,80,39486,62.0,Origin,401,ssrf
2,wWRXeQtO,05/May/2025:09:09:36 +0000,SINGLE_LAYERED,OPTIONS,80,39486,NaN,"Access-Control-Request-Method, Access-Control-...",204,ssrf
3,bg2mL7Uq,05/May/2025:09:09:37 +0000,SINGLE_LAYERED,POST,80,39486,33.0,Origin,200,ssrf
4,KTqEfmTV,05/May/2025:09:09:37 +0000,SINGLE_LAYERED,GET,80,39486,8846.0,Origin,200,ssrf


In [52]:
numeric_cols = [c for c in ["status_code"] if c in df.columns]
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

if "timestamp" in df.columns:
    df["timestamp_dt"] = pd.to_datetime(
        df["timestamp"],
        format="%d/%b/%Y:%H:%M:%S %z",
        errors="coerce"
    )

df = df.reset_index(drop=True)
df.to_csv(Path(data_paths.interim) / "modsec_audit_train_v2.csv")
df.head()

,id,timestamp,layer_type,method,local_port,remote_port,resp_content_length,resp_vary,status_code,target,timestamp_dt
0,XD10tCwH,05/May/2025:09:08:24 +0000,SINGLE_LAYERED,OPTIONS,80,39486,NaN,"Access-Control-Request-Method, Access-Control-...",204,ssrf,2025-05-05 09:08:24+00:00
1,sRNMSqLX,05/May/2025:09:08:25 +0000,SINGLE_LAYERED,POST,80,39486,62.0,Origin,401,ssrf,2025-05-05 09:08:25+00:00
2,wWRXeQtO,05/May/2025:09:09:36 +0000,SINGLE_LAYERED,OPTIONS,80,39486,NaN,"Access-Control-Request-Method, Access-Control-...",204,ssrf,2025-05-05 09:09:36+00:00
3,bg2mL7Uq,05/May/2025:09:09:37 +0000,SINGLE_LAYERED,POST,80,39486,33.0,Origin,200,ssrf,2025-05-05 09:09:37+00:00
4,KTqEfmTV,05/May/2025:09:09:37 +0000,SINGLE_LAYERED,GET,80,39486,8846.0,Origin,200,ssrf,2025-05-05 09:09:37+00:00


In [ ]:
def load_and_clean_audit_csv(
        path: Path,
        keep: List[str] | None = None,
        *,
        duplicate_subset: List[str] | None = None,
) -> pd.DataFrame:
    df = pd.read_csv(
        path,
    )

    df.columns = df.columns.str.strip()
    df = df.rename(columns=str.lower)
